In [26]:
import pandas as pd
import numpy as np
from statistics import mean

import cleaning

In [27]:
test= False

accounts = cleaning.clean_account()
cards = cleaning.clean_card(test)
clients = cleaning.clean_client()
districts = cleaning.clean_district()
loans = cleaning.clean_loans(test)
trans = cleaning.clean_trans(test)
disp = pd.read_csv('../../project/banking_data/disp.csv', sep=';')


C:\Users\utilizador\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


## Creation of a new Transactions Dataframe

The new transactions consists on a dataframe where the transactions from the same account are all summarized in 1 row: `min no. trans`, `max no. trans`, `avg no. trans`, `min_balance`, `max_balance` and `avg_balance`.

In [28]:
# Sort values by account ID to make the job easier
trans = trans.sort_values(by="account_id")

aux = 0

# Create new dataframe with the columns needed
trans_final = pd.DataFrame(columns=["account_id", "no. movements", "min no. trans", "max no. trans", "avg no. trans", "min balance", "max balance", "avg balance"])

for i in loans["account_id"]:
    if i != aux:
        trans_test = trans.loc[trans['account_id'] == i]

        # Get the values for each column of the new dataframe
        no_movements = trans_test.shape[0]
        min_no_trans = min(trans_test["amount"])
        max_no_trans = max(trans_test["amount"])
        avg_no_trans = mean(trans_test["amount"])
        min_balance = min(trans_test["balance"])
        max_balance = max(trans_test["balance"])
        avg_balance = mean(trans_test["balance"])
        
        # Append to the new dataframe
        trans_final = trans_final.append({"account_id": i, "no. movements": no_movements, "min no. trans": min_no_trans, "max no. trans": max_no_trans, "avg no. trans": avg_no_trans, "min balance": min_balance, "max balance": max_balance, "avg balance": avg_balance}, ignore_index=True)
        
        aux = i

## Client-District Merge

Merge the clients with the districts using the `district_id` information.

In [29]:
# Add district columns to the right side of the clients columns 
client_district = clients.merge(districts, left_on="district_id", right_on="code ")

# Drop irrelevant columns
client_district = client_district.drop(columns=["name ", "district_id", "region"])

# Rename necessary columns so we don't have merge conflicts later
client_district = client_district.rename(columns={col:("client_district "+ col) for col in client_district.columns[4:]})

## Account-District Merge

In [30]:
# Add district columns to the right side of the accounts columns 
account_district = accounts.merge(districts, left_on="district_id", right_on="code ")

# Drop irrelevant columns
account_district = account_district.drop(columns=["name ", "district_id", "code ", "region"])

# Rename necessary columns so we don't have merge conflicts later
account_district = account_district.rename(columns={col:("account_district "+ col) for col in account_district.columns[4:]})

## Card-Client Merge

### Useful Renames

In [31]:
# Renamed so it doesn't get any problems while merging with the dispositions
cards = cards.rename(columns={"type":"card_type", "issued":"card_issued"})

### Merging

#### 1. Card-Disposition Merge

In [32]:
# Add disposition columns to the right side of the cards columns 
card_client = cards.merge(disp)

# Drop irrelevant columns
card_client = card_client.drop(columns=["card_id", "disp_id", "client_id", "type"])

# Get Clients that have Loans but don't have a Card
loan_client_no_card = []
for i in loans["account_id"]:
    if not i in list(card_client["account_id"]):
        loan_client_no_card.append(i)

# Get Clients that have a Card but don't have Loans
card_client_no_loan = []
for i in card_client["account_id"]:
    if not i in list(loans["account_id"]):
        card_client_no_loan.append(i)

# Erase account that have cards but don't have a loan
card_client = card_client.drop(index=card_client[card_client["account_id"].isin(card_client_no_loan)].index)

# Add accounts that have loans but don't have a card
loan_client_no_card = np.array([loan_client_no_card, ["None"] * len(loan_client_no_card), [0] * len(loan_client_no_card)]).transpose()
loan_client_no_card = pd.DataFrame(loan_client_no_card, columns=["account_id", "card_type", "card_issued"])

# Add accounts that have loans but don't have a card to the accounts that have loans and have a card
card_client = card_client.append(loan_client_no_card)
card_client["account_id"] = card_client["account_id"].astype(int)

C:\Users\utilizador\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Account-Client Merge

In [33]:
# Merge the disposition columns with the accounts (with district information) columns
account_client = disp.merge(account_district)

# Merge the previous table with the clients (with the district information) columns
account_client = account_client.merge(client_district, on="client_id")

# The only loans that are relevant are the ones that belong to the owner
account_client = account_client[account_client["type"] == "OWNER"]

# Drop irrelevant columns: client id and type (it's always OWNER)
account_client = account_client.drop(columns=["client_id", "type"])

#client_district.sort_values(by="client_id")
account_client

,disp_id,account_id,frequency,date,no. of inhabitants,account_district no. of municipalities with inhabitants < 499,account_district no. of municipalities with inhabitants 500-1999,account_district no. of municipalities with inhabitants 2000-9999,account_district no. of municipalities with inhabitants >10000,account_district no. of cities,...,client_district ratio of urban inhabitants,client_district average salary,client_district unemploymant rate '95,client_district unemploymant rate '96,client_district unemploymant_growth,client_district no. of enterpreneurs per 1000 inhabitants,client_district no. of commited crimes '95,client_district no. of commited crimes '96,client_district crime_growth,client_district total_crime
0,1,1,monthly issuance,24-03-1995,70699,60,13,2,1,4,...,65.3,8968,2.83,3.35,0.52,131,1740.0,1910,170.0,3650.0
1,2,2,monthly issuance,26-02-1993,1204953,0,0,0,1,1,...,100.0,12541,0.29,0.43,0.14,167,85677.0,99107,13430.0,184784.0
3,4,3,monthly issuance,07-07-1997,95616,65,30,4,1,6,...,51.4,9307,3.85,4.43,0.58,118,2616.0,3040,424.0,5656.0
5,6,4,monthly issuance,21-02-1996,107870,84,29,6,1,6,...,58.0,8754,3.83,4.31,0.48,137,3804.0,3868,64.0,7672.0
6,7,5,monthly issuance,30-05-1997,58796,22,16,7,1,5,...,51.9,9045,3.13,3.60,0.47,124,1845.0,1879,34.0,3724.0
7,8,6,monthly issuance,27-09-1994,121947,37,28,7,3,11,...,70.5,8541,2.51,2.97,0.46,131,3496.0,3839,343.0,7335.0
8,9,7,monthly issuance,24-11-1996,110643,49,41,4,1,4,...,51.9,8441,3.45,4.48,1.03,115,1879.0,2252,373.0,4131.0
9,10,8,monthly issuance,21-09-1995,161954,21,37,20,3,8,...,48.0,8720,3.73,4.50,0.77,116,3729.0,3651,-78.0,7380.0
11,12,9,monthly issuance,27-01-1993,285387,0,2,8,5,7,...,85.3,9317,6.49,7.07,0.58,97,6949.0,6872,-77.0,13821.0
12,13,10,monthly issuance,28-08-1996,387570,0,0,0,1,1,...,100.0,9897,1.60,1.96,0.36,140,18721.0,18696,-25.0,37417.0


## Final Merge: Loan

### Useful Rename

In [34]:
loans = loans.rename(columns={"date":"loan_date", "amount":"loan_amount", "duration":"loan_duration"})
trans = trans.rename(columns={"amount":"no. transactions"})
account_client = account_client.rename(columns={"frequency":"account_frequency", "date": "account_creation"})


### Merging

In [35]:
# Merge accounts informations into loans columns
loan_final = loans.merge(account_client, on="account_id")

# Merge cards with clients informations into loans columns
loan_final = loan_final.merge(card_client, on="account_id")

# Merge transactions informations into loans colums
loan_final = loan_final.merge(trans_final, on="account_id")

# Drop irrelevant columns
loan_final = loan_final.drop(columns=["account_id", "disp_id"])


## Add to CSV File

In [36]:

loan_final = loan_final.rename(columns={'status' : 'loan_success'})
loan_final.to_csv("../../csvs/loan_united_"+("test" if test else "train") + ".csv", index=False)
loan_final

,loan_id,loan_date,loan_amount,loan_duration,payments,loan_success,account_frequency,account_creation,no. of inhabitants,account_district no. of municipalities with inhabitants < 499,...,client_district total_crime,card_issued,card_type,no. movements,min no. trans,max no. trans,avg no. trans,min balance,max balance,avg balance
0,5314,05-07-1993,96396,12,8033,-1,weekly issuance,22-03-1993,94812,15,...,5789.0,0,None,4.0,1100.0,9900.0,5025.000000,1100.0,20100.0,12250.000000
1,5316,11-07-1993,165960,36,4610,1,monthly issuance,13-02-1993,112709,48,...,5472.0,0,None,37.0,2.9,54300.0,11015.635135,700.0,120512.8,52083.859459
2,6863,28-07-1993,127080,60,2118,1,monthly issuance,08-02-1993,77917,85,...,4202.0,0,None,24.0,48.6,19065.0,5417.458333,800.0,49590.4,30060.954167
3,5325,03-08-1993,105804,36,2939,1,monthly issuance,30-01-1993,107870,84,...,12899.0,0,None,25.0,14.6,26448.0,8253.080000,1000.0,65898.5,41297.480000
4,7240,06-09-1993,274740,60,4579,1,weekly issuance,14-02-1993,1204953,0,...,3022.0,0,None,27.0,30.0,63366.0,18945.966667,600.0,122893.1,57188.211111
5,6687,13-09-1993,87840,24,3660,1,monthly issuance,12-05-1993,53921,61,...,3472.0,0,None,17.0,51.3,32938.0,9496.929412,500.0,65847.8,46318.552941
6,7284,15-09-1993,52788,12,4399,1,monthly issuance,14-01-1993,58796,22,...,3724.0,0,None,43.0,14.6,17920.0,3714.897674,1000.0,41469.1,22198.179070
7,6111,24-09-1993,174744,24,7281,-1,monthly issuance,14-04-1993,122603,25,...,10471.0,0,None,32.0,14.6,52600.0,11557.950000,1000.0,105628.0,37485.456250
8,7235,13-10-1993,154416,48,3217,1,weekly issuance,20-04-1993,70699,60,...,3650.0,0,None,32.0,13.6,39098.0,9740.150000,8897.0,81723.5,46715.800000
9,5997,04-11-1993,117024,24,4876,1,monthly issuance,20-06-1993,177686,69,...,12899.0,0,None,13.0,45.1,29255.0,14122.576923,800.0,76985.2,44131.169231
